### Sorting and visualizing UniPept lowest common ancestor analyses in Trocas8

#### Beginning with: LCA peptides in csvs

#### Goal: spectral abundance-corrected (NAAF) taxonomic peptide compositions at the 4 stations before and after 24hrs

## Issue: the `join` commands at the end for diatom and bacterial peptides need to be run independently from one another. To do this, run the diatom command, restart the kernal, then run the bacterial. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/analyses/T8-notincs/unipept/

/home/millieginty/Documents/git-repos/amazon/analyses/T8-notincs/unipept


In [4]:
# read the data into pandas dataframes
NMCP_50_GF75_681a = pd.read_csv("lca/cleaned/681_NMCP_50_GF75_lca.csv")

# delete any Metazoa hits because they're trypsin
NMCP_50_GF75_681b = NMCP_50_GF75_681a[NMCP_50_GF75_681a.kingdom != 'Metazoa']

# keep only entries to the phylum level
NMCP_50_GF75_681 = NMCP_50_GF75_681b[NMCP_50_GF75_681b['phylum'].notnull()]

# How many peptides to the phylum level?
print('# of phylum peptides = ', len(NMCP_50_GF75_681))

NMCP_50_GF75_681.head(6)

# of phylum peptides =  8


,peptide,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,...,tribe,subtribe,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma
4,LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,VATVSPLR,Solirubrobacteraceae,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Thermoleophilia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,QEFLNAAK,Phorcysia thermohydrogeniphila,Bacteria,NaN,NaN,NaN,Aquificae,NaN,NaN,Aquificae,...,Phorcysia,NaN,NaN,NaN,Phorcysia thermohydrogeniphila,NaN,NaN,NaN,NaN,NaN
18,DMLGAYK,Eubacteriales,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Clostridia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,VFLSSLTK,Puniceicoccaceae,Bacteria,NaN,NaN,NaN,Verrucomicrobia,NaN,NaN,Opitutae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,TVTVSPLR,Streptomyces,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Actinomycetia,...,Streptomyces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now I want to read in the file containing the stripped peptides with NAAF values
####  - NAAF stands for 'noramlized area abunace factor'

### I want to join the dataframes if they share an index (stripped peptide with equated leucine and isoleucines)
#### - That means I'll reindex the processed peptide file

In [5]:
NMCP_50_GF75_681_NAAFa = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/681_NMCP_50_GF75_DN50_ILnaafs.csv")
NMCP_50_GF75_681_NAAFb = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/681_NMCP_50_GF75_PDB_ILnaafs.csv")
NMCP_50_GF75_681_NAAFc = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/689_NMCP_50_GF75_DN50_ILnaafs.csv")
NMCP_50_GF75_681_NAAFd = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/689_NMCP_50_GF75_PDB_ILnaafs.csv")


frames = [NMCP_50_GF75_681_NAAFa, NMCP_50_GF75_681_NAAFb, NMCP_50_GF75_681_NAAFc, NMCP_50_GF75_681_NAAFd]

NMCP_50_GF75_681_NAAF = pd.concat(frames, sort=False)

NMCP_50_GF75_681_NAAF.set_index('stripped_peptide')
NMCP_50_GF75_681_NAAF = NMCP_50_GF75_681_NAAF.loc[:, ~NMCP_50_GF75_681_NAAF.columns.str.contains('^Unnamed')]

NMCP_50_GF75_681_NAAF.rename(columns = {'stripped_peptide':'peptide'}, inplace = True)

print('# of total peptides = ', len(NMCP_50_GF75_681_NAAF))

print('column names:', NMCP_50_GF75_681_NAAF.columns)

NMCP_50_GF75_681_NAAF.head()

# of total peptides =  248
column names: Index(['peptide', 'Area', 'NAAF_num.', 'stripped_IL'], dtype='object')


,peptide,Area,NAAF_num.,stripped_IL
0,SPATLNSR,62000.0,7750.0,NaN
1,LSSPATLNSR,9990000.0,999000.0,NaN
2,SPATLNSR,62000.0,7750.0,NaN
3,LSSPATLNSR,9990000.0,999000.0,NaN
4,LSSPATLDSR,125000.0,12500.0,NaN


In [6]:
NMCP_50_GF75_681_NAAF['peptide'] = NMCP_50_GF75_681_NAAF['peptide'].astype(str)
NMCP_50_GF75_681['peptide'] = NMCP_50_GF75_681['peptide'].astype(str)

<ipython-input-6-067525068bc3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NMCP_50_GF75_681['peptide'] = NMCP_50_GF75_681['peptide'].astype(str)


In [7]:
# get the intersection of the phylum peptides and corresponding peptides w/ NAAFs

over = np.intersect1d(NMCP_50_GF75_681["peptide"], NMCP_50_GF75_681_NAAF["peptide"])

print(over)

['DMLGAYK' 'LATVLSPR' 'QEFLNAAK' 'VATVSPLR']


In [8]:
# join the dataframes if the peptide values are the same using 'join'
# since a couple are de novo only (more for bacteria), we won't have all the UniPept peptides overlap 

NMCP_50_GF75_681.set_index('peptide', inplace=True)
NMCP_50_GF75_681_NAAF.set_index('peptide', inplace=True)

NMCP_50_GF75_681_Phy = NMCP_50_GF75_681.join(NMCP_50_GF75_681_NAAF, how='left', rsuffix='_other')


print('# of total phylum-level peptides = ', len(NMCP_50_GF75_681_Phy))

NMCP_50_GF75_681_Phy.head()

# of total phylum-level peptides =  23


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,subgenus,species group,species subgroup,species,subspecies,varietas,forma,Area,NAAF_num.,stripped_IL
peptide,,,,,,,,,,,,,,,,,,,,,
DMLGAYK,Eubacteriales,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Clostridia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,315000.0,45000.0,NaN
DMLGAYK,Eubacteriales,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Clostridia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,749000.0,107000.0,NaN
LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6600000.0,825000.0,NaN
LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6600000.0,825000.0,NaN
LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6600000.0,825000.0,NaN


In [9]:
# write to a csv

NMCP_50_GF75_681_Phy.to_csv("lca/NAAF/NMCP_50_GF75_681_Phy_naaf.csv")

!ls lca/NAAF/

BY_50_GF75_675_Phy_naaf.csv    NMCP_50_GF75_681_Phy_naaf.csv
BY_surf_GF75_676_Phy_naaf.csv  SMCP_50_GF75_679_Phy_naaf.csv
CV_50_GF75_677_Phy_naaf.csv    SMCP_surf_GF75_680_Phy_naaf.csv
CV_surf_GF75_678_Phy_naaf.csv
